In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/04 17:39:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.3.2'

In [8]:
!ls -lh data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv.gz

-rw-rw-r-- 1 rohitg rohitg 19M Dec  2  2022 data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv.gz


In [11]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [13]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv.gz')

df = df.repartition(6)

df.write.parquet('data/pq/fhv/2019/10/', mode='overwrite')

In [14]:
!ls -lh data/pq/fhv/2019/10/

total 39M
-rw-r--r-- 1 rohitg rohitg    0 Mar  4 17:54 _SUCCESS
-rw-r--r-- 1 rohitg rohitg 6.4M Mar  4 17:54 part-00000-3714e664-7f0c-4d47-8618-9c672c73995c-c000.snappy.parquet
-rw-r--r-- 1 rohitg rohitg 6.4M Mar  4 17:54 part-00001-3714e664-7f0c-4d47-8618-9c672c73995c-c000.snappy.parquet
-rw-r--r-- 1 rohitg rohitg 6.4M Mar  4 17:54 part-00002-3714e664-7f0c-4d47-8618-9c672c73995c-c000.snappy.parquet
-rw-r--r-- 1 rohitg rohitg 6.4M Mar  4 17:54 part-00003-3714e664-7f0c-4d47-8618-9c672c73995c-c000.snappy.parquet
-rw-r--r-- 1 rohitg rohitg 6.4M Mar  4 17:54 part-00004-3714e664-7f0c-4d47-8618-9c672c73995c-c000.snappy.parquet
-rw-r--r-- 1 rohitg rohitg 6.4M Mar  4 17:54 part-00005-3714e664-7f0c-4d47-8618-9c672c73995c-c000.snappy.parquet


In [15]:
df = spark.read.parquet('data/pq/fhv/2019/10/')

**Q3**: How many taxi trips were there on October 15?

In [16]:
from pyspark.sql import functions as F

In [17]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62610

In [20]:
df.registerTempTable('fhv_2019_10')

/home/rohitg/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [21]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhv_2019_10
WHERE
    to_date(pickup_datetime) = '2019-10-15';
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



**Q4**: Longest trip for each day

In [18]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [19]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2019-10-28|   2272149000|
| 2019-10-11|   2272149000|
| 2019-10-31|    315620787|
| 2019-10-01|    252460901|
| 2019-10-17|     31658400|
+-----------+-------------+



In [33]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2019-10-28|          631152.5|
| 2019-10-11|          631152.5|
| 2019-10-31| 87672.44083333333|
| 2019-10-01| 70128.02805555555|
| 2019-10-17|            8794.0|
| 2019-10-26| 8784.166666666666|
| 2019-10-30|1464.5344444444445|
| 2019-10-25|1056.8266666666666|
| 2019-10-02| 769.2313888888889|
| 2019-10-23| 745.6166666666667|
+-----------+------------------+



**Q5**: Most frequent `dispatching_base_num`

How many stages this spark job has?



In [23]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1)
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B01145|   47548|
|              B00256|   43234|
|              B00856|   36778|
|              B03016|   34985|
|              B01239|   33586|
+--------------------+--------+



In [24]:
df \
    .groupBy('dispatching_base_num') \
        .count() \
    .orderBy('count', ascending=False) \
    .limit(5) \
    .show()

+--------------------+-----+
|dispatching_base_num|count|
+--------------------+-----+
|              B01145|47548|
|              B00256|43234|
|              B00856|36778|
|              B03016|34985|
|              B01239|33586|
+--------------------+-----+



**Q6**: Most common locations pair

In [25]:
df_zones = spark.read.parquet('zones')

In [26]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [27]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [28]:
df_zones.registerTempTable('zones')

In [32]:
spark.sql("""
SELECT
    pul.Zone,
    COUNT(1)
FROM 
    fhv_2019_10 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
GROUP BY 
    1
ORDER BY
    2 ASC
LIMIT 5;
""").show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
+--------------------+--------+

